In [35]:
import requests
import json
import pandas as pd
from pandas import json_normalize

import time
from tqdm import tqdm_notebook
from selenium import webdriver

In [36]:
naver_data = pd.read_csv('./data/meamul_up_to_date_detail.csv')
landbook = pd.read_csv('./data/landbook.csv')

C:\Users\gongdan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (76,97,98,100,101,104,107,108,109,110,275,282,283,285) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
temp = naver_data[['articleDetail.latitude','articleDetail.longitude','articleDetail.pnu']]

In [38]:
temp.drop_duplicates(subset=['articleDetail.latitude','articleDetail.longitude'], inplace= True)
temp = temp.reset_index(drop = True)

C:\Users\gongdan\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [39]:
df = pd.merge(temp, landbook, left_on=['articleDetail.latitude','articleDetail.longitude'], right_on=['naver_latitude','naver_longitude'], how = 'outer')

In [40]:
find_landbook = df[df['id'].isna() == True]
find_landbook = find_landbook[['articleDetail.latitude','articleDetail.longitude','articleDetail.pnu']]
find_landbook.columns = ['naver_latitude','naver_longitude','naver_pnu']

In [41]:
len(find_landbook)

0

In [42]:
result_landbook = []
for idx, key in enumerate(tqdm_notebook(find_landbook.index)):
    url = 'https://api2.landbook.net/point_polygons?lon={lon}&lat={lat}'.format(lon = find_landbook['naver_longitude'][key], lat = find_landbook['naver_latitude'][key])
    r = requests.get(url, timeout = 5)
    result_landbook. append([json.loads(r.text)['id'], find_landbook['naver_longitude'][key], find_landbook['naver_latitude'][key],find_landbook['naver_pnu'][key]])
    time.sleep(0.5)

C:\Users\gongdan\AppData\Local\Temp/ipykernel_6764/3642305530.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, key in enumerate(tqdm_notebook(find_landbook.index)):


0it [00:00, ?it/s]

In [43]:
result_landbook = pd.DataFrame(result_landbook)
result_landbook.columns = ['landbook_pnu','naver_longitude','naver_latitude','naver_pnu']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [ ]:
#data = json_normalize(r.json())

In [ ]:
result_jibeon = pd.DataFrame()
for idx, item in enumerate(tqdm_notebook(result_landbook['landbook_pnu'])):
    
    url = 'https://api2.landbook.net/parcels/{num}'.format(num = item)
    r = requests.get(url, timeout = 5)
    try:
        info = json.loads(r.text)['parcel_info']
        info.pop('geometry')
        info.pop('building_list')
        info.pop('other_parcel')
        info = dict(info)
        temp = pd.DataFrame(info,  index = [0])
        result_jibeon = pd.concat([result_jibeon, temp])
        time.sleep(0.5)
    except:
        print(idx, "error", item)
result_jibeon = result_jibeon.reset_index(drop = True)

In [ ]:
result_jibeon

In [ ]:
result_jibeon = result_jibeon.reset_index(drop = True)

In [ ]:
result_jibeon[['naver_longitude','naver_latitude','naver_pnu']] = result_landbook[['naver_longitude','naver_latitude','naver_pnu']]

In [ ]:
landbook = pd.read_csv('./data/landbook.csv')

In [ ]:
result = pd.concat([landbook, result_jibeon])

In [ ]:
result.drop_duplicates().to_csv('./data/landbook.csv', index = False, encoding = "utf-8-sig")